In [129]:
import vrep
import time
import numpy as np
from scipy import linalg
import math

# Set up and move every joint

# Forward Kinematics

In [41]:
def skew(m):
    return np.array([[0 ,-m[2], m[1]],[m[2], 0, -m[0]], [-m[1], m[0], 0]])


In [60]:
def revolute(a,q):
    ab = skew(a)
    bottom = -1*np.dot(ab,q)
    return np.array([[a[0]],[a[1]],[a[2]],[bottom[0]],[bottom[1]],[bottom[2]]])

In [77]:
def brack6(a):
    ws = a[:3]
    vs = a[3:]
    wb = skew(ws)
    ans = np.array([[wb[0,0], wb[0,1], wb[0,2], vs[0]],[wb[1,0], wb[1,1], wb[1,2], vs[1]],[wb[2,0], wb[2,1], wb[2,2], vs[2]], [0,0,0,0]])
    return ans

In [127]:
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

In [148]:
def forward_kinematics(theta):
    
    a0 = np.array([0,1,0])
    q0 = np.array([0,0,0])
    
    a1 = np.array([-1,0,0])
    q1 = np.array([0,0.152,0])
    
    a2 = np.array([-1,0,0])
    q2 = np.array([-0.12,0.396,0])
    
    a3 = np.array([-1,0,0])
    q3 = np.array([-0.027,0.609,0])
    
    a4 = np.array([0,-1,0])
    q4 = np.array([-0.11,0.609,0])
    
    a5 = np.array([-1,0,0])
    q5 = np.array([-0.11,0.692,0])
    
    M = np.array([[1, 0, 0, -0.93], [0 ,1 ,0 ,0.745], [0, 0, 1, 0], [0 ,0 ,0 ,1]])
    
    axis = [a0,a1,a2,a3,a4,a5]
    point = [q0,q1,q2,q3,q4,q5]
    
    screw = [revolute(axis[a],point[a]) for a in range(len(axis))]
    
    exp = [linalg.expm(brack6(screw[s])*theta[s]) for s in range (len(screw))]
    
    ex_multiplied = np.identity(4)
    for i in exp:
        ex_multiplied = ex_multiplied.dot(i)
    T = ex_multiplied.dot(M)
    return T

In [144]:
a = np.array([1,2,3])
q = np.array([4,5,6])
rev = revolute(a,q)

theta = np.array([1,1,1,1,1,1])
print (forward_kinematics(theta))

[[ 0.67450028 -0.55844195  0.48289953 -1.03663731]
 [ 0.70807342  0.67450028 -0.20900097 -0.30483211]
 [-0.20900097  0.48289953  0.85036853  0.03622021]
 [ 0.          0.          0.          1.        ]]


In [149]:
vrep.simxFinish(-1)

# Connect to V-REP (raise exception on failure)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
if clientID == -1:
    raise Exception('Failed connecting to remote API server')

# Start simulation
vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

# Wait two seconds
time.sleep(2)


result, joint_one_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint1', vrep.simx_opmode_blocking)
result, joint_two_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint2', vrep.simx_opmode_blocking)
result, joint_three_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint3', vrep.simx_opmode_blocking)
result, joint_four_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint4', vrep.simx_opmode_blocking)
result, joint_five_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint5', vrep.simx_opmode_blocking)
result, joint_six_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint6', vrep.simx_opmode_blocking)

result, dummyhandle=vrep.simxCreateDummy(clientID,.1,None,vrep.simx_opmode_blocking)

theta = np.repeat(np.pi/3,6)
T = forward_kinematics(theta)
R = T[:3,:3]
p = T[:3,3]
euler = rotationMatrixToEulerAngles(R)

vrep.simxSetObjectPosition(clientID,dummyhandle,-1,(p[0],p[1],p[2]),vrep.simx_opmode_blocking)
vrep.simxSetObjectOrientation(clientID,dummyhandle,-1,(euler[0],euler[1],euler[2]),vrep.simx_opmode_blocking)

theta=np.pi/3
vrep.simxPauseCommunication(clientID,True)
vrep.simxSetJointTargetPosition(clientID, joint_one_handle, theta, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_two_handle, theta, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_three_handle, theta, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_four_handle, theta, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_five_handle, theta, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_six_handle, theta, vrep.simx_opmode_oneshot)

vrep.simxPauseCommunication(clientID,False)

# time.sleep(1)
# theta=0
# vrep.simxPauseCommunication(clientID,True)
# vrep.simxSetJointTargetPosition(clientID, joint_one_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_two_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_three_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_four_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_five_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_six_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxPauseCommunication(clientID,False)
# time.sleep(1)

# time.sleep(1)
# theta=-np.pi/3
# vrep.simxPauseCommunication(clientID,True)
# vrep.simxSetJointTargetPosition(clientID, joint_one_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_two_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_three_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_four_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_five_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxSetJointTargetPosition(clientID, joint_six_handle, theta, vrep.simx_opmode_oneshot)
# vrep.simxPauseCommunication(clientID,False)

time.sleep(5)


# Stop simulation
vrep.simxStopSimulation(clientID, vrep.simx_opmode_oneshot)

# Before closing the connection to V-REP, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
vrep.simxGetPingTime(clientID)

# Close the connection to V-REP
vrep.simxFinish(clientID)
